In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

使用Vertex AI和BigQuery ML预测零售需求

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/demand_forecasting/forecasting-retail-demand.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/demand_forecasting/forecasting-retail-demand.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/workbench/demand_forecasting/forecasting-retail-demand.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI工作台中打开
    </a>
  </td>                                                                                               
</table>

## 概述

本教程探讨如何使用BigQuery公共零售数据集进行需求预测。能够衡量和预测顾客需求可以帮助零售商更好地了解他们的顾客，储存适当的产品，提供定向促销，并且一般来说，更好地规划和管理预算。本文档在零售数据上应用了BigQuery ML上的ARIMA（自回归积分滑动平均）模型。本文档演示了如何训练和评估用于需求预测数据集的BigQuery ML模型，并提取可行的未来见解。

*注意：此笔记本文件设计为在[Vertex AI Workbench托管笔记本](https://cloud.google.com/vertex-ai/docs/workbench/managed/create-instance)实例中使用`Python (Local)`内核运行。此笔记本的某些组件可能无法在其他笔记本环境中运行。*

#### 使用BigQuery ML的ARIMA建模

<a href='https://en.wikipedia.org/wiki/Autoregressive_integrated_moving_average'>ARIMA模型</a>旨在分析历史数据，发现随时间变化的模式，并将其推断到未来—即预测。该模型在BigQuery ML中可用，使用户能够使用SQL查询直接在BigQuery中创建和执行机器学习模型。使用BigQuery ML具有优势，因为它已经可以访问数据，如果需要的话，它可以自动处理大部分建模细节，并且还会将模型和任何预测结果存储在BigQuery中。

了解更多关于[Vertex AI Workbench](https://cloud.google.com/vertex-ai/docs/workbench/introduction)和[BigQuery ML](https://cloud.google.com/bigquery-ml/docs/managing-models-vertex)。

### 目标
在本教程中，您将学习如何在 BigQuery ML 上针对零售数据构建 ARIMA（自回归积分移动平均）模型。

本教程使用以下 Google Cloud ML 服务：
- BigQuery

执行的步骤包括：

* 探索数据
* 使用 BigQuery 和 ARIMA 模型建模
* 评估模型
* 使用 BigQuery ML 评估模型结果（在训练数据上）
* 评估模型结果 - MAE，MAPE，MSE，RMSE（在测试数据上）
* 使用执行者功能

### 数据集

这个笔记本使用BigQuery公共零售数据集。数据涵盖了10家美国商店，包括商品级别、部门、产品类别和商店详细信息。此外，它还具有价格和毛利等解释变量。

成本
本教程使用谷歌云的以下收费组件：

- Vertex AI
- BigQuery

了解[Vertex AI的定价](https://cloud.google.com/vertex-ai/pricing)，[BigQuery的定价](https://cloud.google.com/bigquery/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成成本估算。

安装额外的软件包。

In [ ]:
! pip3 install --quiet --upgrade pandas-gbq 'google-cloud-bigquery[bqstorage,pandas]' scikit-learn

只有Colab：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### 开始之前

#### 设置您的项目 ID

**如果您不知道您的项目 ID**，请尝试以下操作：
- 运行 `gcloud config list`
- 运行 `gcloud projects list`
- 查看支持页面: [查找项目 ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# set the project id
! gcloud config set project $PROJECT_ID

### 地区

您还可以更改 Vertex AI 使用的 `REGION` 变量。
了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

### 对您的Google Cloud账户进行身份验证

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关说明进行操作。

**1. Vertex AI 工作站**
- 不需要进行任何操作，因为您已经验证通过了。

**2. 本地JupyterLab实例**，将注释取消并运行。

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账户或其他
- 在这里查看所有的认证选项：[Google Cloud 平台 Jupyter Notebook 认证指南](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/notebook_authentication_guide.ipynb)

### 创建一个云存储桶

创建一个存储桶，用于存储诸如数据集等中间产物。

In [ ]:
BUCKET_NAME = "your-bucket-name-unique"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

只有当您的存储桶不存在时，才运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

UUID

如果您正在进行实时教程会话，您可能正在使用共享的测试帐户或项目。为了避免在创建资源时发生用户之间的名称冲突，您可以为每个实例会话创建一个UUID，并将其附加到您在本教程中创建的资源的名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

导入库并定义常量

加载所需的库。

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from google.cloud import bigquery
from google.cloud.bigquery import Client
from sklearn.metrics import (mean_absolute_error,
                             mean_absolute_percentage_error,
                             mean_squared_error)

%matplotlib inline

import warnings

warnings.filterwarnings("ignore")

为表设置名称

In [ ]:
SALES_TABLE = "training_data_table"

# Construct a BigQuery client object.

client = Client(project=PROJECT_ID)

创建一个BigQuery数据集

In [ ]:
dataset_id = "demandforecasting" + "_" + UUID

如果您正在使用***Vertex AI Workbench托管笔记本实例***，每个以"#@bigquery"开头的单元格将被视为一个SQL查询。如果您在使用Vertex AI Workbench用户管理的笔记本实例或Colab，则它将被视为一个标记单元格。

#@bigquery
CREATE SCHEMA [your-dataset-id]
OPTIONS(
  location="us"
  ) 

大家注意
CREATE SCHEMA [你的数据集ID]
选项(
  位置="美国"
  )

（可选）如果您正在使用Vertex AI Workbench管理的笔记本实例，一旦从BigQuery显示在下面的单元格中的结果，点击“查询和加载为DataFrame”按钮，并执行生成的代码存根以将数据获取到当前笔记本中作为一个数据框。

注意：默认情况下，数据会加载到一个名为`df`的变量中，但如果需要的话，可以在执行单元格之前更改它。

In [ ]:
query = """
CREATE SCHEMA `{PROJECT_ID}.{dataset_id}`
OPTIONS(
  location="us"
  )
""".format(
    PROJECT_ID=PROJECT_ID, dataset_id=dataset_id
)
query_job = client.query(query)
print(query_job.result())

## 探索数据
查看存储在公共BigQuery数据集中的数据。

#@bigquery
SELECT * FROM `looker-private-demo.retail.transaction_detail`

In [ ]:
query = """
SELECT * FROM `looker-private-demo.retail.transaction_detail`
"""
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

创建一个名为`important_fields`的视图，只使用`transaction_timestamp`和`line_items`字段，其中商店ID为10。

#@bigquery
CREATE OR REPLACE VIEW [your-dataset-id].important_fields AS
(
    SELECT transaction_timestamp, line_items FROM `looker-private-demo.retail.transaction_detail` WHERE store_id = 10
)

In [ ]:
query = """
CREATE OR REPLACE VIEW {dataset_id}.important_fields AS
(
    SELECT transaction_timestamp,line_items from `looker-private-demo.retail.transaction_detail` WHERE store_id = 10
)   
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)
print(query_job.result())

查看“important_fields”视图中的数据。

#@bigquery
SELECT * FROM [你的数据集ID].important_fields

In [ ]:
query = """
SELECT * FROM {dataset_id}.important_fields
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

query_job.to_dataframe()

将`transaction_timestamp`字段转换为日期。

#@bigquery 
创建或替换视图[your-dataset-id].data_after_converting_timestamp_to_date AS 
(
SELECT EXTRACT(DATE FROM transaction_timestamp AT TIME ZONE "UTC") AS date,line_items from [your-dataset-id].important_fields
)

In [ ]:
query = """
CREATE OR REPLACE VIEW {dataset_id}.data_after_converting_timestamp_to_date AS
(
    SELECT EXTRACT(DATE FROM transaction_timestamp AT TIME ZONE "UTC") AS date,line_items from {dataset_id}.important_fields
)   
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)
print(query_job.result())

查看数据并检查`date`字段的数值。

#@bigquery
SELECT * FROM [your-dataset-id].data_after_converting_timestamp_to_date

#@bigquery
从[your-dataset-id].data_after_converting_timestamp_to_date选择*

In [ ]:
query = """
SELECT * FROM {dataset_id}.data_after_converting_timestamp_to_date
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

query_job.to_dataframe()

将数据加载到一个数据框中。

In [ ]:
df_intermediary = query_job.to_dataframe()

检查数据帧字段的数据类型。

In [ ]:
df_intermediary.dtypes

`line_items` 字段是一个结构数组。将数组拆分为其各个部分，`product_id`，`sale_price`和`gross_margin`。

#@bigquery
CREATE OR REPLACE VIEW [your-dataset-id].split_array_of_structs AS

(SELECT 日期, line_items
FROM [your-dataset-id].data_after_converting_timestamp_to_date, UNNEST(line_items) AS line_items)

In [ ]:
query = """
CREATE OR REPLACE VIEW {dataset_id}.split_array_of_structs AS
 
(SELECT date,line_items
FROM {dataset_id}.data_after_converting_timestamp_to_date, UNNEST(line_items) AS line_items)
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)
print(query_job.result())

查看数据。

#@bigquery
SELECT * FROM [your-dataset-id].split_array_of_structs

In [ ]:
query = """
SELECT * FROM {dataset_id}.split_array_of_structs
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

query_job.to_dataframe()

移除额外的列，只保留`日期`和`产品编号`。

#@bigquery
CREATE OR REPLACE VIEW [your-dataset-id].splitting_struct_columns AS
 
(SELECT date,line_items.product_id as product_id
FROM [your-dataset-id].split_array_of_structs) 

#@大查询
创建或替换视图[your-dataset-id].splitting_struct_columns AS

（选择日期，line_items.product_id作为 product_id
从[your-dataset-id].split_array_of_structs中）

In [ ]:
query = """
CREATE OR REPLACE VIEW {dataset_id}.splitting_struct_columns AS
 
(SELECT date,line_items.product_id as product_id
FROM {dataset_id}.split_array_of_structs)
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)
print(query_job.result())

查看数据。

#@bigquery
SELECT * FROM [your-dataset-id].splitting_struct_columns

In [ ]:
query = """
SELECT * FROM {dataset_id}.splitting_struct_columns 
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

query_job.to_dataframe()

统计每天产品的销售量。

#@bigquery
CREATE OR REPLACE VIEW [your-dataset-id].sales_count_per_date AS
 
(SELECT date,product_id,COUNT(*) as sales_count
FROM [your-dataset-id].splitting_struct_columns GROUP BY date,product_id) 

将上述英文文本翻译成中文：
#@bigquery
创建或替换视图[your-dataset-id]。sales_count_per_date AS
 
（选择日期，产品ID， COUNT(*) 作为销售量
从[your-dataset-id].splitting_struct_columns 中 GROUP BY 日期，产品ID）

In [ ]:
query = """
CREATE OR REPLACE VIEW {dataset_id}.sales_count_per_date AS
 
(SELECT date,product_id,COUNT(*) as sales_count
FROM {dataset_id}.splitting_struct_columns GROUP BY date,product_id)
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)
print(query_job.result())

#@bigquery
从[your-dataset-id].sales_count_per_date中选择*

In [ ]:
query = """
SELECT * FROM {dataset_id}.sales_count_per_date
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

query_job.to_dataframe()

为在整个日期范围内销售数量最多的五种产品创建视图。

#@bigquery
CREATE OR REPLACE VIEW [your-dataset-id].top_five_products AS (
    WITH topsellingitems AS(
         SELECT 
            product_id,
            sum(sales_count) sum_sales
        FROM
            `[your-dataset-id].sales_count_per_date` 
        GROUP BY 
            product_id
        ORDER BY sum_sales DESC
        LIMIT 5 #Top N
    )
    SELECT 
        date,
        product_id,
        sales_count
    FROM
        `[your-dataset-id].sales_count_per_date` 
    WHERE
        product_id IN (SELECT product_id FROM topsellingitems)
    )

In [ ]:
query = """
CREATE OR REPLACE VIEW {dataset_id}.top_five_products AS (
    WITH topsellingitems AS(
         SELECT 
            product_id,
            sum(sales_count) sum_sales
        FROM
            `{dataset_id}.sales_count_per_date` 
        GROUP BY 
            product_id
        ORDER BY sum_sales DESC
        LIMIT 5 #Top N
    )
    SELECT 
        date,
        product_id,
        sales_count
    FROM
        `{dataset_id}.sales_count_per_date` 
    WHERE
        product_id IN (SELECT product_id FROM topsellingitems)
    )
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)
print(query_job.result())

#@bigquery
SELECT * FROM [你的数据集ID].top_five_products

In [ ]:
query = """
SELECT * FROM {dataset_id}.top_five_products
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

query_job.to_dataframe()

将数据加载到数据框中并查看数据。

In [ ]:
df = query_job.to_dataframe()
print(df)

检查你的数据框中的字段数据类型。

In [ ]:
df.dtypes

将`date`字段的数据类型转换为`datetime`。

In [ ]:
df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d")

对于这个预测模型，每个产品的所有日期都需要有日期值。

为了构建一个数据帧，对于产品没有销售的日期应该用 `0` 值填充 `sales_count` 字段，确定最小和最大日期，以便知道哪些日期需要 `0` 值。

**首先，获取最早的（最小）日期。**

#@bigquery
SELECT MIN(DATE) FROM [your-dataset-id].top_five_products

In [ ]:
query = """
SELECT MIN(DATE) AS min_date FROM {dataset_id}.top_five_products
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

query_job.to_dataframe()

获取最新的（最大的）日期。

#@bigquery
SELECT MAX(DATE) FROM [your-dataset-id].top_five_products

中文翻译：#@bigquery
SELECT MAX(DATE) FROM [your-dataset-id].top_five_products

In [ ]:
query = """
SELECT MAX(DATE) FROM {dataset_id}.top_five_products
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

query_job.to_dataframe()

将完整的日期范围值添加到数据帧中。

In [ ]:
dates = pd.date_range(start="2016-12-17", end="2021-10-06").to_frame()

获取`dates`数据框的描述。

In [ ]:
dates.info()

查看其中一个产品的数据，按日期排序，以显示数据集中许多日期并不存在。

In [ ]:
df.loc[df["product_id"] == 20552].sort_values(by=["date"])

要创建一个数据框，其中每个产品都没有缺失日期，请将五个产品的数据与`dates`数据框合并。

从`product_id` `20552`开始。

In [ ]:
df1 = (
    pd.merge(
        df.loc[df["product_id"] == 20552],
        dates,
        left_on="date",
        right_on=0,
        how="outer",
    )
    .sort_values(by=["date"])
    .drop(columns=0)
)  # merging dates dataframe with product_id matching rows
df1["product_id"] = 20552  # product_id will be null so making it the specified values
df1.reset_index(inplace=True, drop=True)  # making index to start from 0
df1 = df1.fillna(0)  # for sales_count making null values as 0
df1["sales_count"] = df1["sales_count"].astype(
    "int"
)  # convert sales_count column to integer
print("data after converting for a product with product_id 20552")
print(df1)

df2 = (
    pd.merge(
        df.loc[df["product_id"] == 13596],
        dates,
        left_on="date",
        right_on=0,
        how="outer",
    )
    .sort_values(by=["date"])
    .drop(columns=0)
)  # merging dates dataframe with product_id matching rows
df2["product_id"] = 13596  # product_id will be null so making it the specified values
df2.reset_index(inplace=True, drop=True)  # making index to start from 0
df2 = df2.fillna(0)  # for sales_count making null values as 0
df2["sales_count"] = df2["sales_count"].astype(
    "int"
)  # convert sales_count column to integer
print(df2)

df3 = (
    pd.merge(
        df.loc[df["product_id"] == 23641],
        dates,
        left_on="date",
        right_on=0,
        how="outer",
    )
    .sort_values(by=["date"])
    .drop(columns=0)
)  # merging dates dataframe with product_id matching rows
df3["product_id"] = 23641  # product_id will be null so making it the specified values
df3.reset_index(inplace=True, drop=True)  # making index to start from 0
df3 = df3.fillna(0)  # for sales_count making null values as 0
df3["sales_count"] = df3["sales_count"].astype(
    "int"
)  # convert sales_count column to integer
print(df3)

df4 = (
    pd.merge(
        df.loc[df["product_id"] == 28305],
        dates,
        left_on="date",
        right_on=0,
        how="outer",
    )
    .sort_values(by=["date"])
    .drop(columns=0)
)  # merging dates dataframe with product_id matching rows
df4["product_id"] = 28305  # product_id will be null so making it the specified values
df4.reset_index(inplace=True, drop=True)  # making index to start from 0
df4 = df4.fillna(0)  # for sales_count making null values as 0
df4["sales_count"] = df4["sales_count"].astype(
    "int"
)  # convert sales_count column to integer
print(df4)

df5 = (
    pd.merge(
        df.loc[df["product_id"] == 20547],
        dates,
        left_on="date",
        right_on=0,
        how="outer",
    )
    .sort_values(by=["date"])
    .drop(columns=0)
)  # merging dates dataframe with product_id matching rows
df5["product_id"] = 20547  # product_id will be null so making it the specified values
df5.reset_index(inplace=True, drop=True)  # making index to start from 0
df5 = df5.fillna(0)  # for sales_count making null values as 0
df5["sales_count"] = df5["sales_count"].astype(
    "int"
)  # convert sales_count column to integer
print(df5)

将所有五个数据框合并成一个新的数据框。

In [ ]:
pdList = [df1, df2, df3, df4, df5]  # List of your dataframes
new_df = pd.concat(pdList)
new_df

重置数据帧的索引。

In [ ]:
new_df.reset_index(inplace=True, drop=True)
print(new_df)

查看五个产品ID。

#@bigquery
SELECT DISTINCT product_id from [your-dataset-id].top_five_products

查询唯一的product_id从[你的数据集编号].top_five_products

In [ ]:
query = """
SELECT DISTINCT product_id from {dataset_id}.top_five_products
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

在产品销售数量随时间变化的图表中绘制“销售数量”。

In [ ]:
plt.plot(
    new_df.loc[new_df["product_id"] == 20552]["date"],
    new_df.loc[new_df["product_id"] == 20552]["sales_count"],
)
plt.xticks(rotation="vertical")

In [ ]:
plt.plot(
    new_df.loc[new_df["product_id"] == 20547]["date"],
    new_df.loc[new_df["product_id"] == 20547]["sales_count"],
)
plt.xticks(rotation="vertical")

In [ ]:
plt.plot(
    new_df.loc[new_df["product_id"] == 28305]["date"],
    new_df.loc[new_df["product_id"] == 28305]["sales_count"],
)
plt.xticks(rotation="vertical")

In [ ]:
plt.plot(
    new_df.loc[new_df["product_id"] == 23641]["date"],
    new_df.loc[new_df["product_id"] == 23641]["sales_count"],
)
plt.xticks(rotation="vertical")

In [ ]:
plt.plot(
    new_df.loc[new_df["product_id"] == 13596]["date"],
    new_df.loc[new_df["product_id"] == 13596]["sales_count"],
)
plt.xticks(rotation="vertical")

列出 `new_df` 数据框的数据类型。

In [ ]:
new_df.dtypes

将`new_df`数据框创建为一个新的BigQuery表。

In [ ]:
job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.
    schema=[
        bigquery.SchemaField("product_id", bigquery.enums.SqlTypeNames.INTEGER),
        bigquery.SchemaField("date", bigquery.enums.SqlTypeNames.DATE),
        bigquery.SchemaField("sales_count", bigquery.enums.SqlTypeNames.INTEGER),
    ],
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition="WRITE_TRUNCATE",
)

# save the dataframe to a table in the created dataset
job = client.load_table_from_dataframe(
    new_df,
    "{}.{}.{}".format(PROJECT_ID, dataset_id, SALES_TABLE),
    job_config=job_config,
)  # Make an API request.
job.result()  # Wait for the job to complete.

通过设置日期范围来创建一个训练数据集，限制所使用的数据。

In [ ]:
# select the date-range and item-id(top 5) for training-data and create a table for the same
TRAININGDATA_STARTDATE = "2016-12-17"
TRAININGDATA_ENDDATE = "2021-6-01"
query = """
CREATE OR REPLACE TABLE {PROJECT_ID}.{DATASET}.training_data AS (
    SELECT
        *
    FROM
        `{DATASET}.{SALES_TABLE}`
    WHERE
        date BETWEEN '{STARTDATE}' AND '{ENDDATE}'
        );
""".format(
    STARTDATE=TRAININGDATA_STARTDATE,
    ENDDATE=TRAININGDATA_ENDDATE,
    DATASET=dataset_id,
    SALES_TABLE=SALES_TABLE,
    PROJECT_ID=PROJECT_ID,
)
# execute the query (as it is a create query, there won't be any tabular output)
query_job = client.query(query)
print(query_job.result())

选择用于绘图的原始数据。

In [ ]:
df_historical = new_df[
    (new_df["date"] >= pd.to_datetime(TRAININGDATA_STARTDATE))
    & (new_df["date"] <= pd.to_datetime(TRAININGDATA_ENDDATE))
].copy()
df_historical

使用BigQuery和ARIMA模型进行建模

**使用训练数据创建一个ARIMA模型。**

#@bigquery
CREATE OR REPLACE MODEL [your-dataset-id].arima_model

OPTIONS(
  MODEL_TYPE='ARIMA',
  TIME_SERIES_TIMESTAMP_COL='日期', 
  TIME_SERIES_DATA_COL='销售数量',
  TIME_SERIES_ID_COL='产品编号',
  HOLIDAY_REGION='美国'
    
) AS

SELECT 
    日期,
    产品编号,
    销售数量
FROM
  [your-dataset-id].training_data

训练ARIMA模型。

In [ ]:
# Train an ARIMA model on the created dataset
query = """
CREATE OR REPLACE MODEL `{PROJECT_ID}.{DATASET}.arima_model`

OPTIONS(
  MODEL_TYPE='ARIMA',
  TIME_SERIES_TIMESTAMP_COL='date',
  TIME_SERIES_DATA_COL='sales_count',
  TIME_SERIES_ID_COL='product_id') AS

SELECT
    date,
    product_id,
    sales_count
FROM
  `{DATASET}.training_data`
""".format(
    PROJECT_ID=PROJECT_ID, DATASET=dataset_id
)
# execute the query
job = client.query(query)
job.result()

评估模型

要评估已训练的模型，请在最后一天的训练数据之后的90天获取预测。在BigQuery ML中，使用`HORIZON`参数提供预测天数。使用`CONFIDENCE_LEVEL`参数指定预测的置信区间。

#@bigquery dfforecast 

DECLARE HORIZON STRING DEFAULT "90";
DECLARE CONFIDENCE_LEVEL STRING DEFAULT "0.90";

EXECUTE IMMEDIATE format('''
    SELECT
      *
    FROM
      ML.FORECAST(MODEL [your-dataset-id].arima_model,
                  STRUCT(%s AS horizon,
                         %s AS confidence_level)
                 )
    ''',HORIZON,CONFIDENCE_LEVEL) 

请将上述英文文本翻译成中文。

将数据加载到名为`dfforecast`的数据框中。

In [ ]:
query = '''DECLARE HORIZON STRING DEFAULT "90"; #number of values to forecast
DECLARE CONFIDENCE_LEVEL STRING DEFAULT "0.90"; ## required confidence level

EXECUTE IMMEDIATE format("""
    SELECT
      *
    FROM
      ML.FORECAST(MODEL {dataset_id}.arima_model,
                  STRUCT(%s AS horizon,
                         %s AS confidence_level)
                 )
    """,HORIZON,CONFIDENCE_LEVEL)'''.format(
    dataset_id=dataset_id
)
job = client.query(query)
dfforecast = job.to_dataframe()

查看前几行。

In [ ]:
dfforecast.head()

In [ ]:
print(f"Number of rows: {dfforecast.shape[0]}")

清理历史和预测值以进行绘图。

In [ ]:
df_historical.sort_values(by=["product_id", "date"], inplace=True)
dfforecast.sort_values(by=["product_id", "forecast_timestamp"], inplace=True)

# Select the actual data to plot against the forecasted data
day_diff = (new_df["date"] - pd.to_datetime(TRAININGDATA_ENDDATE)).dt.days
df_actual_90d = new_df[new_df["product_id"].isin(dfforecast["product_id"].unique())][
    (day_diff > 0) & (day_diff <= 90)
].copy()
df_actual_90d.shape

绘制历史和预测数据。

In [ ]:
def plot_hist_forecast(
    historical, forecast, actual, hist_start="", hist_end="", title=""
):
    if hist_start != "":
        historical = historical[
            historical["date"] >= pd.to_datetime(hist_start, format="%Y-%m-%d")
        ].copy()
    if hist_end != "":
        historical = historical[
            historical["date"] <= pd.to_datetime(hist_end, format="%Y-%m-%d")
        ].copy()

    plt.figure(figsize=(15, 4))
    plt.plot(historical["date"], historical["sales_count"], label="historical")
    # Plot the forecast data
    plt.plot(
        forecast["forecast_timestamp"],
        forecast["forecast_value"],
        label="forecast",
        linestyle="--",
    )
    # Plot the actual data
    plt.plot(actual["date"], actual["sales_count"], label="actual")
    # plot the confidence interval
    confidence_level = forecast["confidence_level"].iloc[0] * 100
    low_CI = forecast["confidence_interval_lower_bound"]
    upper_CI = forecast["confidence_interval_upper_bound"]

    # Shade the confidence interval
    plt.fill_between(
        forecast["forecast_timestamp"],
        low_CI,
        upper_CI,
        color="#539caf",
        alpha=0.4,
        label=f"{confidence_level} confidence interval",
    )
    plt.legend()
    plt.title(title)
    plt.show()
    return


product_id_list = dfforecast["product_id"].unique()
for i in product_id_list:
    print("Product_id : ", i)
    plot_hist_forecast(
        df_historical[df_historical["product_id"] == i],
        dfforecast[dfforecast["product_id"] == i],
        df_actual_90d[df_actual_90d["product_id"] == i],
        hist_start="2021-02-01",
        title=i,
    )

大多数预测结果看起来还不错，您也可以看到实际范围落在模型建议的90%置信区间内。在内部，BigQuery ML执行许多计算昂贵的任务，即使考虑了季节性和假日信息。

BigQuery ML的ARIMA模型学到的系数也可以通过查询模型中的<a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-arima-coefficients">ARIMA_COEFFICIENTS</a> 进行检查。

#@bigquery
选择
  *
从
  ML.ARIMA_COEFFICIENTS(MODEL [your-dataset-id].arima_model)

In [ ]:
query = """
SELECT
  *
FROM 
  ML.ARIMA_COEFFICIENTS(MODEL {dataset_id}.arima_model)
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

在上述结果中，
- <b>product_id</b>列代表我们在训练ARIMA模型时指定的索引列。
- <b>ar_coefficients</b>列对应于ARIMA算法中的自回归系数（非季节性p）。
- <b>ma_coefficients</b>指的是ARIMA算法中的移动平均系数（非季节性q）。
- <b>intercept_or_drift</b>是ARIMA算法中的常数项。

使用BigQuery ML评估模型结果

BigQuery ML 还提供了 <a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-evaluate">ML.EVALUATE</a> 函数，用于检查训练模型的评估指标。对于 ARIMA 模型，您可以查看模型在 `log_likelihood`, `AIC` 和 `variance` 上的评估情况。

#@bigquery
选择
  *
从
  ML.评估(模型 [你的数据集id].arima_model)

In [ ]:
query = """
SELECT
  *
FROM
  ML.EVALUATE(MODEL {dataset_id}.arima_model)
""".format(
    dataset_id=dataset_id
)
query_job = client.query(query)

In [ ]:
query_job.to_dataframe()

## 评估模型结果 - MAE，MAPE，MSE，RMSE（在测试数据上）

通常，要评估预测模型，您可以根据您想要评估的方式选择指标。首先，您可以从以下选项中选择：
* <b>平均绝对误差（MAE）</b>：实际值和预测值之间的绝对差值的平均值。
* <b>平均百分比误差（MAPE）</b>：实际值和预测值之间的绝对差值与实际值之间的百分比的平均值。
* <b>均方误差（MSE）</b>：实际值和预测值之间的平方差值的平均值。
* <b>均方根误差（RMSE）</b>：MSE的平方根。

In [ ]:
df_actual_90d.sort_values(by=["product_id", "date"], inplace=True)
df_actual_90d.reset_index(drop=True, inplace=True)
dfforecast.sort_values(by=["product_id", "forecast_timestamp"], inplace=True)
dfforecast.reset_index(drop=True, inplace=True)

In [ ]:
errors = {"product_id": [], "MAE": [], "MAPE": [], "MSE": [], "RMSE": []}
for i in product_id_list:
    mae = mean_absolute_error(
        df_actual_90d[df_actual_90d["product_id"] == i]["sales_count"],
        dfforecast[dfforecast["product_id"] == i]["forecast_value"],
    )
    mape = mean_absolute_percentage_error(
        df_actual_90d[df_actual_90d["product_id"] == i]["sales_count"],
        dfforecast[dfforecast["product_id"] == i]["forecast_value"],
    )

    mse = mean_squared_error(
        df_actual_90d[df_actual_90d["product_id"] == i]["sales_count"],
        dfforecast[dfforecast["product_id"] == i]["forecast_value"],
        squared=True,
    )

    rmse = mean_squared_error(
        df_actual_90d[df_actual_90d["product_id"] == i]["sales_count"],
        dfforecast[dfforecast["product_id"] == i]["forecast_value"],
        squared=False,
    )

    errors["product_id"].append(i)
    errors["MAE"].append(mae)
    errors["MAPE"].append(mape)
    errors["MSE"].append(mse)
    errors["RMSE"].append(rmse)
errors = pd.DataFrame(errors)
errors

从这些误差指标获得的数值来看，产品ID 20552的13,596的误差指标较高。请注意，这些误差指标是在测试期间所有个别预测的总和，因此反映了模型在选定时期内的整体表现。理想情况下，这些误差指标越低，模型的预测能力就越好。

## 托管实例中的执行器功能

如果您正在使用托管实例，在笔记本上方的顶部工具栏中，点击**执行器**按钮。

<img src="images/navbar_exe.png" ></img>的中文翻译是：<img src="images/navbar_exe.png" ></img>

给执行命名。选择**云存储桶**，**机器类型**和**加速器类型**。对于**环境**，请选择**Python 3**。

<img src="images/exe_form.png" style="width:500px"></img>

将上述英文文本翻译为中文：<img src="images/exe_form.png" style="width:500px"></img>

<img src="images/python3_env_selection.png" style="width:500px"></img> 的中文翻译： <img src="images/python3_env_selection.png" style="width:500px"></img>

在“类型”菜单中，您可以选择基于日程安排的重复执行，如果您想要安排执行按照时间间隔运行，比如每小时一次。

<img src="images/schedule.png" style="height:200px;"></img> 的中文翻译是：<img src="images/schedule.png" style="height:200px;"></img>

您可以通过点击左侧边栏上的**笔记本执行者**按钮来查看您执行的历史记录。看起来像一个日历图标。

<img src="images/side_nav.png" style="height:500px;"></img> 的中文翻译是：<img src="images/side_nav.png" style="height:500px;"></img>

您可以在 **Executions** 选项卡中看到执行历史。

将上述英文文本翻译成中文：<img src="images/list_execution.png" style="width:500px"></img>

您可以在“日程表”选项卡上查看您的活动日程。

<img src="images/list_schedule.png" style="width:500px"></img> 的中文翻译是：<img src="images/list_schedule.png" style="width:500px"></img>

整理清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。以下代码将删除整个数据集。

In [ ]:
# Set dataset variable to the ID of the dataset to fetch.
dataset = f"{PROJECT_ID}.{dataset_id}"

# Use the delete_contents parameter to delete a dataset and its contents.
# Use the not_found_ok parameter to not receive an error if the dataset has already been deleted.
client.delete_dataset(
    dataset_id, delete_contents=True, not_found_ok=True
)  # Make an API request.

print("Deleted dataset '{}'.".format(dataset_id))